# Decoder-Vorhersage + Softmax & Cross-Entropy (Update für $W_{out}$)

Dieses Notebook entspricht den Folien:

## Teil 1: Decoder-Vorhersage
Kontextvektor $h$ → Logits $z$ → Softmax $p$ → Token-ID (argmax) → Token (Tokenizer-Rückübersetzung)

## Teil 2: Training-Update für $W_{out}$
Softmax + Cross-Entropy, inkl. Gradient:
\[
\frac{\partial L}{\partial z} = p - y
\qquad
\frac{\partial L}{\partial W_{out}} = (p-y)\,x^T
\]

Wir reproduzieren exakt die Zahlen aus **Bsp. 1** und **Bsp. 2** auf deiner Folie.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)


## Hilfsfunktionen (Softmax, Cross-Entropy, One-Hot, Gradienten)

In [ ]:
def softmax(z: np.ndarray) -> np.ndarray:
    """Softmax ohne Stabilisierung, damit e^z und p wie in der Folie nachvollziehbar bleiben."""
    e = np.exp(z)
    return e / np.sum(e)

def one_hot(k: int, n: int) -> np.ndarray:
    y = np.zeros(n, dtype=float)
    y[k] = 1.0
    return y

def cross_entropy(p: np.ndarray, y: np.ndarray) -> float:
    """CE für One-Hot: L = -sum(y_i log p_i)."""
    eps = 1e-12
    return float(-np.sum(y * np.log(p + eps)))

def grad_logits(p: np.ndarray, y: np.ndarray) -> np.ndarray:
    """dL/dz = p - y"""
    return p - y

def grad_W_out(p: np.ndarray, y: np.ndarray, x: np.ndarray) -> np.ndarray:
    """dL/dW = (p-y) x^T"""
    return np.outer(p - y, x)


# Teil 1 – Decoder-Vorhersage (Logits → Softmax → Token-ID → Token)

Wir verwenden ein Mini-Vokabular (so wie auf deiner Folie):
- 12856 → "Frankreich"
- 5312  → "Deutschland"
- 9021  → "Spanien"

Dann nehmen wir beispielhafte Logits und lassen das Modell per Softmax entscheiden.

In [ ]:
id_to_token = {
    12856: "Frankreich",
    5312:  "Deutschland",
    9021:  "Spanien"
}

token_ids = list(id_to_token.keys())
tokens = [id_to_token[i] for i in token_ids]

# Beispiel-Logits (kannst du beliebig ändern)
z = np.array([3.2, 1.7, 1.2], dtype=float)  # [Frankreich, Deutschland, Spanien]
p = softmax(z)

pred_idx = int(np.argmax(p))
pred_token_id = token_ids[pred_idx]
pred_token = id_to_token[pred_token_id]

df_pred = pd.DataFrame({
    "Token-ID": token_ids,
    "Token": tokens,
    "Logit z": z,
    "Softmax p": p
})

display(df_pred)
print("\nVorhersage (argmax):")
print("  Token-ID:", pred_token_id)
print("  Token:", pred_token)


In [ ]:
plt.figure(figsize=(7, 3.5))
plt.bar(tokens, p)
plt.title("Decoder: Softmax-Wahrscheinlichkeiten")
plt.ylabel("p")
plt.ylim(0, float(np.max(p) * 1.25))
plt.grid(axis="y", linestyle="--", linewidth=0.5)
for i, v in enumerate(p):
    plt.text(i, float(v) + 0.01, f"{float(v):.2f}", ha="center", va="bottom")
plt.show()


# Teil 2 – Softmax & Cross-Entropy (Update für $W_{out}$)

Wir reproduzieren die Folienbeispiele:

## Bsp. 1 (3 Klassen)
- Logits: $z=[2.0, 1.0, 0.1]$
- Softmax: $p=[0.6590, 0.2424, 0.0986]$
- Ziel (One-Hot): $y=[1,0,0]$ (Klasse 1 = "Frankreich")
- Input-Vektor (aus Attention): $x=[1.5, 0.5]$

Gradienten:
- $\frac{\partial L}{\partial z} = p-y = [-0.3410, 0.2424, 0.0986]$
- $\frac{\partial L}{\partial W} = (p-y) x^T$ → Matrix wie auf der Folie


In [ ]:
# ---- Bsp. 1 (wie Folie) ----
z1 = np.array([2.0, 1.0, 0.1], dtype=float)
p1 = softmax(z1)
y1 = np.array([1.0, 0.0, 0.0], dtype=float)
x1 = np.array([1.5, 0.5], dtype=float)

dL_dz1 = grad_logits(p1, y1)
dL_dW1 = grad_W_out(p1, y1, x1)
L1 = cross_entropy(p1, y1)

df1 = pd.DataFrame({
    "z": z1,
    "p": p1,
    "y": y1,
    "p-y": dL_dz1
}, index=["Klasse 1 (Frankreich)", "Klasse 2 (Deutschland)", "Klasse 3 (Spanien)"])

print("Bsp. 1 – Loss L = -log(p_frankreich)")
print("p_frankreich =", round(float(p1[0]), 4))
print("L =", round(L1, 3))
display(df1)

dfW1 = pd.DataFrame(dL_dW1, columns=["x1", "x2"], index=["Klasse 1", "Klasse 2", "Klasse 3"]) 
print("Gradient dL/dW = (p-y) x^T (soll Folienwerten entsprechen):")
display(dfW1.round(4))


### Check gegen Folie (Bsp. 1)

Erwartete Gradientenmatrix (Folie):
- Klasse 1: [-0.5115, -0.1705]
- Klasse 2: [ 0.3636,  0.1212]
- Klasse 3: [ 0.1479,  0.0493]


In [ ]:
expected_W1 = np.array([
    [-0.5115, -0.1705],
    [ 0.3636,  0.1212],
    [ 0.1479,  0.0493]
])

print("Max. Abweichung (|Notebook - Folie|):", float(np.max(np.abs(dL_dW1 - expected_W1))))


## Bsp. 2 (4 Klassen)

- Logits: $z=[3.52, 2.37, 1.185, 0.07]$
- Softmax: $p=[0.692, 0.219, 0.066, 0.022]$
- Ziel (One-Hot): $y=[1,0,0,0]$ (Klasse 1 = "Frankreich")
- Input-Vektor: $x=[1.2, 0.8, 0.5]$

Gradient:
\[
\frac{\partial L}{\partial W} = (p-y) x^T
\]
soll die 4×3 Matrix aus deiner Folie ergeben.

In [ ]:
# ---- Bsp. 2 (wie Folie) ----
z2 = np.array([3.52, 2.37, 1.185, 0.07], dtype=float)
p2 = softmax(z2)
y2 = np.array([1.0, 0.0, 0.0, 0.0], dtype=float)
x2 = np.array([1.2, 0.8, 0.5], dtype=float)

dL_dz2 = grad_logits(p2, y2)
dL_dW2 = grad_W_out(p2, y2, x2)
L2 = cross_entropy(p2, y2)

df2 = pd.DataFrame({
    "z": z2,
    "p": p2,
    "y": y2,
    "p-y": dL_dz2
}, index=["Klasse 1 (Frankreich)", "Klasse 2 (Deutschland)", "Klasse 3 (Spanien)", "Klasse 4 (Italien)"])

print("Bsp. 2 – Loss L = -log(p_frankreich)")
print("p_frankreich =", round(float(p2[0]), 3))
print("L =", round(L2, 3))
display(df2)

dfW2 = pd.DataFrame(dL_dW2, columns=["x1", "x2", "x3"], index=["Klasse 1", "Klasse 2", "Klasse 3", "Klasse 4"]) 
print("Gradient dL/dW = (p-y) x^T:")
display(dfW2.round(4))


### Check gegen Folie (Bsp. 2)

Erwartete Matrix (Folie):
- Klasse 1: [-0.3696, -0.2464, -0.154]
- Klasse 2: [ 0.2628,  0.1752,  0.1095]
- Klasse 3: [ 0.0792,  0.0528,  0.033]
- Klasse 4: [ 0.0264,  0.0176,  0.011]


In [ ]:
expected_W2 = np.array([
    [-0.3696, -0.2464, -0.1540],
    [ 0.2628,  0.1752,  0.1095],
    [ 0.0792,  0.0528,  0.0330],
    [ 0.0264,  0.0176,  0.0110]
])

print("Max. Abweichung (|Notebook - Folie|):", float(np.max(np.abs(dL_dW2 - expected_W2))))


# Zusatz: Mini-Demo eines Updates (nur zur Intuition)

Die Folie sagt: Wenn das Modell besser wird, steigt z. B. $p(Frankreich)$ (z. B. auf 0.85) und der Loss sinkt.

Hier zeigen wir das numerisch über die Loss-Funktion (ohne ein komplettes echtes Modelltraining zu simulieren).

In [ ]:
p_old = 0.6590
L_old = -np.log(p_old)

p_new = 0.85
L_new = -np.log(p_new)

print("Alt:  p_frankreich =", p_old, "-> L =", round(float(L_old), 3))
print("Neu:  p_frankreich =", p_new, "-> L =", round(float(L_new), 3))
print("Loss wurde kleiner, weil -log(p) bei größerem p sinkt.")


In [ ]:
# Visualisierung: -log(p) für p in (0,1)
ps = np.linspace(0.01, 0.999, 200)
Ls = -np.log(ps)

plt.figure(figsize=(7, 3.5))
plt.plot(ps, Ls)
plt.title("Cross-Entropy für korrektes Token: L = -log(p)")
plt.xlabel("p (Wahrscheinlichkeit für korrektes Token)")
plt.ylabel("Loss")
plt.grid(True, linestyle="--", linewidth=0.5)
plt.show()
